In [1]:
import os.path
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import pandas as pd
import dlib
import numpy as np
import copy
import pickle

Loading mode, get test_img names etc.

In [2]:
# input labels.
label = pd.read_csv("labels/fairface_label_val.csv")
imgs_path = "fairface-img-margin025-trainval/val"
img_names = [os.path.join(imgs_path, x) for x in os.listdir(imgs_path)]
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

model_fair_7 = torchvision.models.resnet34(pretrained=True)
model_fair_7.fc = nn.Linear(model_fair_7.fc.in_features, 9)
# if runs on a machine without GPU, should do map_location=torch.device('cpu')
#model_fair_7.load_state_dict(torch.load('fair_face_models/fairface_alldata_20191111.pt')) 
model_fair_7.load_state_dict(torch.load('fair_face_models/my_fairface_model_biased.pt'))
model_fair_7 = model_fair_7.to(device)


# trans

trans = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

/home/clydechx/FairFace/env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/clydechx/FairFace/env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Start running the model.

In [3]:
# # Define a hook for all ReLU layers to compute the sum of activations of each filter
# activation = {}
# def relu_hook(module, input, output, layer_name):
#     activation[layer_name] = output.detach()
# # Register the hook for all ReLU layers
# for layer_name, layer in model.named_modules():
#     if isinstance(layer, torch.nn.ReLU):
#         layer.register_forward_hook(relu_hook, layer_name=layer_name)
num_filters = {}
activation = {}
filter_sums = {}
def get_activation(name):
    def hook(model, input, output):
        # Squeeze the tensor to remove the dimension with size 1
        squeezed_tensor = torch.squeeze(output.detach())
        activation[name] = squeezed_tensor
        # If squeezed_tensor has more than one dimension, flatten it
        if len(squeezed_tensor.shape) > 1:
            flattened_tensor = torch.flatten(squeezed_tensor, start_dim=1)
        else:
            flattened_tensor = squeezed_tensor
        # Sum along the second dimension if it exists, otherwise keep the tensor as is
        if flattened_tensor.dim() > 1:
            summed_tensor = flattened_tensor.sum(dim=1)
        else:
            summed_tensor = flattened_tensor
        # Convert the tensor to a list
        summed_list = summed_tensor.tolist()
        filter_sums[name] = summed_list
        num_filters[name] = len(summed_list)
    return hook

for name, layer in model_fair_7.named_modules():
    if isinstance(layer, torch.nn.ReLU):
        layer.register_forward_hook(get_activation(name))
    

In [4]:
# Define class labels for CIFAR-10
classes = ['Black+Female', 'Black+Male', 'East Asian+Female', 'East Asian+Male', 'Indian+Female', 'Indian+Male', 'Latino_Hispanic+Female', 'Latino_Hispanic+Male', 'Middle Eastern+Female', 'Middle Eastern+Male', 'Southeast Asian+Female', 'Southeast Asian+Male', 'White+Female', 'White+Male']
# Define variables to track accuracy and class-wise accuracy
correct = 0
total = 0
class_correct = [0] * 14
class_total = [0] * 14



# in FairFace model we have the following map.
race_dict = {
            0: 'White',
            1: 'Black',
            2: 'Latino_Hispanic',
            3: 'East Asian',
            4: 'Southeast Asian',
            5: 'Indian',
            6: 'Middle Eastern'
        }
gender_dict = { 0:'Male' , 1 :'Female'}
# this is a map from race and gender to class.
my_class_map = {
    (1, 1): 0,
    (1, 0): 1,
    (3, 1): 2,
    (3, 0): 3,
    (5, 1): 4,
    (5, 0): 5,
    (2, 1): 6,
    (2, 0): 7,
    (6, 1): 8,
    (6, 0): 9,
    (4, 1): 10,
    (4, 0): 11,
    (0, 1): 12,
    (0, 0): 13
}

image_predictation_result = {cl: {} for cl in range(len(classes))}
activation_all_images = []
filtersum_all_images = []
miss_predicated_images = []
# Set the model to evaluation mode
model_fair_7.eval()
my_label = pd.read_csv("labels/fairface_label_val.csv")
for index, img_name in enumerate(img_names):
    if index % 1000 == 0:
        print("Predicting... {}/{}".format(index, len(img_names)))
    img_name_in_label =  'val/' +img_name.split('/')[-1]
    label_row = my_label[my_label['file'] == img_name_in_label].iloc[0]
    
    # load image, run model.
    image = dlib.load_rgb_image(img_name)
    image = trans(image)
    image = image.view(1, 3, 224, 224)  # reshape image to match model dimensions (1 batch size)
    image = image.to(device)
    
    outputs = model_fair_7(image)
    outputs = outputs.cpu().detach().numpy()
    outputs = np.squeeze(outputs)
    
    race_outputs = outputs[:7]
    gender_outputs = outputs[7:9]
    
    race_score = np.exp(race_outputs) / np.sum(np.exp(race_outputs))
    gender_score = np.exp(gender_outputs) / np.sum(np.exp(gender_outputs))

    race_pred = np.argmax(race_score)
    gender_pred = np.argmax(gender_score)

    predict_class = my_class_map[(race_pred,gender_pred)]
    label_class_name = label_row['race']+"+" + label_row['gender']
    label_class = classes.index(label_class_name)
    
    
    total +=1
    class_total[label_class] += 1
    image_predictation_result[label_class ][index] = predict_class
    filtersum_all_images.append(copy.deepcopy(filter_sums))
    
    # predict correctly 
    if predict_class == label_class:
        correct+=1
        class_correct[label_class]+=1
    # predict wrongly
    else:
        miss_predicated_images.append(index)
        print("Predicted label: {}, Ground truth: {}".format(predict_class, label_class))


Predicting... 0/10954
Predicted label: 3, Ground truth: 11
Predicted label: 9, Ground truth: 5
Predicted label: 2, Ground truth: 10
Predicted label: 1, Ground truth: 0
Predicted label: 8, Ground truth: 12
Predicted label: 9, Ground truth: 13
Predicted label: 3, Ground truth: 11
Predicted label: 5, Ground truth: 10
Predicted label: 3, Ground truth: 11
Predicted label: 11, Ground truth: 3
Predicted label: 9, Ground truth: 5
Predicted label: 11, Ground truth: 7
Predicted label: 10, Ground truth: 2
Predicted label: 5, Ground truth: 1
Predicted label: 2, Ground truth: 4
Predicted label: 10, Ground truth: 2
Predicted label: 2, Ground truth: 8
Predicted label: 12, Ground truth: 8
Predicted label: 7, Ground truth: 11
Predicted label: 7, Ground truth: 5
Predicted label: 6, Ground truth: 4
Predicted label: 12, Ground truth: 6
Predicted label: 8, Ground truth: 4
Predicted label: 12, Ground truth: 8
Predicted label: 13, Ground truth: 5
Predicted label: 12, Ground truth: 13
Predicted label: 10, Gro

In [5]:
# Print the overall accuracy and accuracy for each class
print('size of test set: {}, accuracy of the network on the test set: {:.2f}%'.format(total, 100 * correct / total))
for i in range(14):
    print('Accuracy of {} : {:.0f}% of {} images'.format(classes[i], 100 * class_correct[i] / class_total[i], class_total[i]))
print("miss_predicated_images num = {}".format(len(miss_predicated_images)))

size of test set: 10954, accuracy of the network on the test set: 59.38%
Accuracy of Black+Female : 60% of 757 images
Accuracy of Black+Male : 54% of 799 images
Accuracy of East Asian+Female : 73% of 773 images
Accuracy of East Asian+Male : 73% of 777 images
Accuracy of Indian+Female : 71% of 763 images
Accuracy of Indian+Male : 54% of 753 images
Accuracy of Latino_Hispanic+Female : 44% of 830 images
Accuracy of Latino_Hispanic+Male : 38% of 793 images
Accuracy of Middle Eastern+Female : 51% of 396 images
Accuracy of Middle Eastern+Male : 69% of 813 images
Accuracy of Southeast Asian+Female : 53% of 680 images
Accuracy of Southeast Asian+Male : 40% of 735 images
Accuracy of White+Female : 72% of 963 images
Accuracy of White+Male : 68% of 1122 images
miss_predicated_images num = 4449


In [6]:
print(len(activation_all_images))
# print(len(activation_all_images[0]))
# print(len(activation_all_images[0]["features.22"][0]))
hooked_layer_names = list(activation.keys())
print(hooked_layer_names)
print(len(filtersum_all_images[0]))
print((filtersum_all_images[0]))

0
['relu', 'layer1.0.relu', 'layer1.1.relu', 'layer1.2.relu', 'layer2.0.relu', 'layer2.1.relu', 'layer2.2.relu', 'layer2.3.relu', 'layer3.0.relu', 'layer3.1.relu', 'layer3.2.relu', 'layer3.3.relu', 'layer3.4.relu', 'layer3.5.relu', 'layer4.0.relu', 'layer4.1.relu', 'layer4.2.relu']
17
{'relu': [6902.42333984375, 2055.44384765625, 3193.74609375, 4554.61669921875, 2075.437744140625, 3282.76806640625, 2260.494140625, 12588.0166015625, 6679.41796875, 392.167236328125, 3840.67041015625, 4153.439453125, 1049.033203125, 2715.30810546875, 3239.101318359375, 0.0, 0.0, 2818.8515625, 2271.50830078125, 151.23858642578125, 4537.07470703125, 3511.21826171875, 2741.09619140625, 660.6498413085938, 4321.2470703125, 2959.79638671875, 7638.47607421875, 8522.2158203125, 4451.451171875, 2973.71435546875, 6282.3154296875, 0.0, 3167.60302734375, 5770.0576171875, 258.6809997558594, 0.0, 5115.1357421875, 3565.72802734375, 9445.8720703125, 2063.2041015625, 4025.4951171875, 0.0, 3077.183349609375, 656.4389038085

In [7]:
def save_intermediate_data(data, file_path):
    """
    Save intermediate data to a file using pickle serialization.

    Args:
        data: The data to be saved.
        file_path (str): The path of the file to save the data.
    """
    with open(file_path, 'wb') as file:
        pickle.dump(data, file)
# save intermediate results
save_intermediate_data(activation_all_images, "activation_CIFAR10_VGG16.pkl")
save_intermediate_data(image_predictation_result, "image_predictation_result.pkl")
save_intermediate_data(miss_predicated_images, "miss_predicated_images.pkl")
save_intermediate_data(hooked_layer_names, "hooked_layer_names.pkl")
save_intermediate_data(filtersum_all_images, "filtersum_all_images.pkl")
save_intermediate_data(num_filters, "num_filters.pkl")

In [8]:
def load_intermediate_data(file_path):
    """
    Load intermediate data from a file.

    Args:
        file_path (str): The path of the file to load the data from.

    Returns:
        The loaded data.
    """
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return data


activation_all_images = load_intermediate_data("activation_CIFAR10_VGG16.pkl")
image_predictation_result = load_intermediate_data("image_predictation_result.pkl")
miss_predicated_images = load_intermediate_data("miss_predicated_images.pkl")
hooked_layer_names = load_intermediate_data("hooked_layer_names.pkl")
filtersum_all_images = load_intermediate_data("filtersum_all_images.pkl")
num_filters = load_intermediate_data("num_filters.pkl")

In [9]:
num_test_images = len(filtersum_all_images)
def find_outliers_zscore(data, threshold=1):
    """
    Find outliers in a list of values using z-score method.

    Args:
        data (list or numpy array): List of numeric values.
        threshold (float, optional): Z-score threshold for identifying outliers. Default is 3.

    Returns:
        list: List of outlier values.
    """
    # Convert data to numpy array
    data = np.array(data)
    if np.std(data) == 0:
        return []
    # Calculate z-scores
    z_scores = (data - np.mean(data)) / np.std(data)
    # Find outliers based on the threshold
    # outliers = data[np.abs(z_scores) > threshold]
    outliers = np.where(z_scores < -threshold)[0]
    return outliers

In [10]:
# find outlier for all mis_predicated images:
# cat (3)
class_id = 3
all_image_class = image_predictation_result[class_id].keys()
wrong_images = [x for x in all_image_class if image_predictation_result[class_id][x] != class_id]
print("num of wrong images in class {} is {}".format(class_id, len(wrong_images)))

num of wrong images in class 3 is 213


In [11]:
z_score_threshold = 1.0

for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    for image in wrong_images:
        # print("image # {}".format(image))
        outliers = find_outliers_zscore(filtersum_all_images[image][layer_name], threshold=z_score_threshold)
        if len(outliers) > 0:
            print("image # {}, layer # {} has outliers {}".format(image, layer_name, outliers))



layer relu
image # 19, layer # relu has outliers [12 15 16 23 31 35 41 46 47 48 49 57 63]
image # 171, layer # relu has outliers [15 16 31 34 35 41 45 46 47 48 49 57 60 63]
image # 250, layer # relu has outliers [12 15 16 19 23 31 35 41 45 46 47 48 49 57 60 63]
image # 280, layer # relu has outliers [12 15 16 19 23 31 34 35 41 45 46 47 48 49 50 57 60 63]
image # 349, layer # relu has outliers [12 15 16 19 23 31 35 41 45 46 47 48 49 50 57 60 63]
image # 399, layer # relu has outliers [15 16 19 31 34 35 41 43 46 47 49 57 60 63]
image # 519, layer # relu has outliers [15 16 19 23 31 34 35 41 45 46 47 48 49 50 57 60 63]
image # 540, layer # relu has outliers [12 15 16 23 31 35 41 46 47 48 49 57 60 63]
image # 551, layer # relu has outliers [15 16 19 31 34 35 41 45 46 47 49 57 60 63]
image # 738, layer # relu has outliers [ 9 15 16 19 31 34 35 41 45 46 47 49 57 60 63]
image # 739, layer # relu has outliers [12 15 16 23 31 35 41 46 47 48 49 50 57 60 63]
image # 780, layer # relu has outliers

## activation level for the whole test set


In [12]:
print(num_filters)

{'relu': 64, 'layer1.0.relu': 64, 'layer1.1.relu': 64, 'layer1.2.relu': 64, 'layer2.0.relu': 128, 'layer2.1.relu': 128, 'layer2.2.relu': 128, 'layer2.3.relu': 128, 'layer3.0.relu': 256, 'layer3.1.relu': 256, 'layer3.2.relu': 256, 'layer3.3.relu': 256, 'layer3.4.relu': 256, 'layer3.5.relu': 256, 'layer4.0.relu': 512, 'layer4.1.relu': 512, 'layer4.2.relu': 512}


In [13]:
# outliers for each layer for each image
z_score_threshold = 2
all_outliers_for_image = {x: {} for x in hooked_layer_names}
num_times_being_outlier = {}

for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    num_times_being_outlier[layer_name] = [0] * num_filters[layer_name]
    for image in range(num_test_images):
        all_outliers_for_image[layer_name][image] = find_outliers_zscore(filtersum_all_images[image][layer_name], threshold=z_score_threshold)
        for k in all_outliers_for_image[layer_name][image]:
            num_times_being_outlier[layer_name][k] += 1

layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu


In [14]:
non_activation_ratio_whole_data = {}
ratio_threshold = 0.01
for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    s = {x: num_times_being_outlier[layer_name][x] / num_test_images for x in range(num_filters[layer_name]) if num_times_being_outlier[layer_name][x] > 0}
    non_activation_ratio_whole_data[layer_name] = {k: v for k, v in s.items() if v > ratio_threshold}
    # non_activation_ratio_whole_data[layer_name] = [x/num_test_images for x in num_times_being_outlier[layer_name]]



layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu


In [15]:
for layer_name in hooked_layer_names:
    print(layer_name, non_activation_ratio_whole_data[layer_name])



relu {}
layer1.0.relu {}
layer1.1.relu {}
layer1.2.relu {39: 0.028391455176191346}
layer2.0.relu {}
layer2.1.relu {}
layer2.2.relu {80: 0.013784918751141136}
layer2.3.relu {105: 0.01990140587913091}
layer3.0.relu {7: 0.015519444951615849, 84: 0.011320065729413912, 121: 0.03313857951433266, 122: 0.010681029760817966, 231: 0.023644330838050027, 251: 0.03186050757714077}
layer3.1.relu {18: 0.0234617491327369, 111: 0.14496987401862332, 118: 0.07905787840058426, 139: 0.022183677195545006, 144: 0.02802629176556509, 146: 0.18732882965126893, 151: 0.1127442030308563, 161: 0.020449150995070293, 162: 0.1258900858134015, 180: 0.18951981011502647, 197: 0.10498448055504839, 213: 0.10334124520723023, 229: 0.0350556874201205, 231: 0.05760452802629176, 236: 0.06043454445864525, 251: 0.29751688880774146}
layer3.2.relu {15: 0.017162680299433998, 18: 0.01606719006755523, 111: 0.011411356582070476, 118: 0.10105897389081614, 139: 0.01935366076319153, 151: 0.11721745481102794, 162: 0.1329194814679569, 182: 

## non-activation times ratio for different classes, for right/wrong

In [16]:
# outliers for each layer for each image
z_score_threshold = 2
all_outliers_for_miss_predicated_image = {x: {} for x in hooked_layer_names}
num_times_being_outlier_miss_predicated_images = {}

for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    num_times_being_outlier_miss_predicated_images[layer_name] = [0] * num_filters[layer_name]
    for image in wrong_images:
        all_outliers_for_miss_predicated_image[layer_name][image] = find_outliers_zscore(filtersum_all_images[image][layer_name], threshold=z_score_threshold)
        for k in all_outliers_for_miss_predicated_image[layer_name][image]:
            num_times_being_outlier_miss_predicated_images[layer_name][k] += 1

layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu


In [17]:
non_activation_ratio_miss_predicated_image = {}
num_miss_predicated_images = len(wrong_images)

for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    s = {x: num_times_being_outlier_miss_predicated_images[layer_name][x] / num_miss_predicated_images for x in range(num_filters[layer_name]) if num_times_being_outlier_miss_predicated_images[layer_name][x] > 0}
    non_activation_ratio_miss_predicated_image[layer_name] = {k: v for k, v in s.items() if v > ratio_threshold}


for layer_name in hooked_layer_names:
    print(layer_name, non_activation_ratio_miss_predicated_image[layer_name])



layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu
relu {}
layer1.0.relu {}
layer1.1.relu {}
layer1.2.relu {39: 0.04225352112676056}
layer2.0.relu {}
layer2.1.relu {}
layer2.2.relu {31: 0.014084507042253521}
layer2.3.relu {76: 0.014084507042253521, 105: 0.018779342723004695}
layer3.0.relu {7: 0.018779342723004695, 22: 0.014084507042253521, 241: 0.023474178403755867, 251: 0.03286384976525822}
layer3.1.relu {18: 0.014084507042253521, 111: 0.107981220657277, 118: 0.07042253521126761, 139: 0.018779342723004695, 144: 0.03755868544600939, 146: 0.12206572769953052, 151: 0.056338028169014086, 162: 0.056338028169014086, 180: 0.13615023474178403, 197: 0.12206572769953052, 213: 0.07981220657276995, 229: 0.0328638497652582

## non-activation times for different classes

In [18]:
all_outliers_for_classes = {x: {y: {} for y in range(len(classes))} for x in hooked_layer_names}
num_times_being_outlier_classes = {y: {} for y in range(len(classes))}

for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    for cla in range(len(classes)):
        num_times_being_outlier_classes[cla][layer_name] = [0] * num_filters[layer_name]
        for image in image_predictation_result[cla]:
            all_outliers_for_classes[layer_name][cla][image] = find_outliers_zscore(filtersum_all_images[image][layer_name], threshold=z_score_threshold)
            for k in all_outliers_for_classes[layer_name][cla][image]:
                num_times_being_outlier_classes[cla][layer_name][k] += 1



layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu


In [19]:
non_activation_ratio_classes = {y: {} for y in range(len(classes))}
ratio_threshold = 0.01
for cla in range(len(classes)):
    for layer_name in hooked_layer_names:
        print("layer {}".format(layer_name))
        s = {x: num_times_being_outlier_classes[cla][layer_name][x] / len(image_predictation_result[cla]) for x in range(num_filters[layer_name]) if num_times_being_outlier_classes[cla][layer_name][x] > 0}
        non_activation_ratio_classes[cla][layer_name] = {k: v for k, v in s.items() if v > ratio_threshold}



layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu
layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu
layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu
layer r

In [20]:
# for cla in range(len(classes)):
#     for layer_name in hooked_layer_names:
#         print(layer_name, non_activation_ratio_classes[cla][layer_name])

# less non-activation classes: 0, 2, 4, 8
# many: 1, 3, 5, 6, 7, 9
cla = 9
print('Accuracy of {} : {:.0f}% of {} images'.format(classes[cla], 100 * class_correct[cla] / class_total[cla], class_total[cla]))
for layer_name in hooked_layer_names:
    print(layer_name, non_activation_ratio_classes[cla][layer_name])


Accuracy of Middle Eastern+Male : 69% of 813 images
relu {}
layer1.0.relu {}
layer1.1.relu {}
layer1.2.relu {39: 0.033210332103321034}
layer2.0.relu {}
layer2.1.relu {1: 0.01968019680196802}
layer2.2.relu {80: 0.012300123001230012}
layer2.3.relu {76: 0.017220172201722016, 105: 0.04059040590405904}
layer3.0.relu {7: 0.03567035670356704, 84: 0.017220172201722016, 121: 0.03690036900369004, 122: 0.02214022140221402, 231: 0.02706027060270603, 241: 0.012300123001230012, 251: 0.03198031980319803}
layer3.1.relu {18: 0.014760147601476014, 111: 0.14514145141451415, 118: 0.12423124231242312, 122: 0.013530135301353014, 139: 0.017220172201722016, 144: 0.012300123001230012, 146: 0.2607626076260763, 151: 0.0959409594095941, 161: 0.020910209102091022, 162: 0.18450184501845018, 172: 0.015990159901599015, 180: 0.22263222632226323, 197: 0.13038130381303814, 213: 0.13776137761377613, 229: 0.03690036900369004, 231: 0.05043050430504305, 236: 0.08241082410824108, 251: 0.3886838868388684}
layer3.2.relu {1: 0.